In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd
import time

available item files:

https://github.com/wi3jmu/DSS/tree/main/tutorial_notebooks/08_knapsack/data

In [2]:
file = '14_31181'

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/wi3jmu/DSS/main/tutorial_notebooks/08_knapsack/data/'+ file + '.csv')
capacity = int(file.split('_')[1])

In [4]:
data.head()

,key,value,weight
0,0,1945,4990
1,1,321,1142
2,2,2945,7390
3,3,4136,10372
4,4,1107,3114


In [5]:
print(capacity)

31181


In [6]:
def knapsack_mip(items, capacity):
    
    start_time = time.time()
    
    weights = items.weight
    values = items.value
    items_list = items.index.tolist()

    m = LpProblem("KnapsackProblem", LpMaximize)

    # Variables
    x = LpVariable.dicts('x', items_list, lowBound=0, upBound=1, cat='binary')

    # Objective
    m += lpSum([values[i] * x[i] for i in items_list])

    # Constraints
    m += LpAffineExpression(list(map(tuple,(zip(x.values(), weights))))) <= capacity

    m.solve(PULP_CBC_CMD(msg=False))
    
    taken = [var.varValue for var in m.variables()]
    
    duration = time.time() - start_time
    
    return (m.objective.value(), duration)

In [7]:
def knapsack_heuristic(items, capacity):
    
    start_time = time.time()
   
    items['density'] = items['value'] / items['weight']

    value = 0
    weight = 0
    taken = [0]*len(items)

    items = items.sort_values('density', ascending=False)

    for i, item in items.iterrows():
        if weight + item.weight <= capacity:
            taken[i] = 1
            value += item.value
            weight += item.weight
               
    duration = time.time() - start_time
            
    return (value, duration)

In [8]:
results = {}
results['Heuristik'] = knapsack_heuristic(data, capacity)
results['MIP'] = knapsack_mip(data, capacity)
pd.DataFrame(results)

,Heuristik,MIP
0,11981.000000,12901.542114
1,0.007816,0.077575


In [9]:
percent = 0.9
items = data.sort_values('value', ascending=False).iloc[:int(len(data)*percent)]

In [10]:
results = {}
results['Heuristik'] = knapsack_heuristic(items, capacity)
results['MIP'] = knapsack_mip(items, capacity)
pd.DataFrame(results)

,Heuristik,MIP
0,11981.000000,12901.542114
1,0.003135,0.022863
